In [20]:
import glob
import json
import numpy as np

np.set_printoptions(suppress=True)

In [21]:
#files = glob.glob('./dat/*timeline*')
files = glob.glob('./dat_challenger_only/*timeline*')

print(len(files))

7539


In [4]:
def calculate_team_total_and_difference(dat, frames):
    team_total = np.array([np.sum(dat[:,:5],axis=1),np.sum(dat[:,5:],axis=1)]).transpose()
    team_total_diff = np.concatenate((team_total,(team_total[:,0] - team_total[:,1]).reshape((frames,1))),axis=1)
    return team_total_diff

def calculate_team_max_and_difference(dat, frames):
    team_max = np.array([np.max(dat[:,:5],axis=1),np.max(dat[:,5:],axis=1)]).transpose()
    team_max_diff = np.concatenate((team_max,(team_max[:,0] - team_max[:,1]).reshape((frames,1))),axis=1)
    return team_max_diff

def get_dat(match):
    current_gold = []
    total_gold = []
    xp = []
    frames = len(match['frames'])
    for frame in range(frames):
        frame_current_gold = []
        frame_total_gold = []
        frame_xp = []
        for i in range(1,11):
            frame_current_gold.append(match['frames'][frame]['participantFrames'][str(i)]['currentGold'])
            frame_total_gold.append(match['frames'][frame]['participantFrames'][str(i)]['totalGold'])
            frame_xp.append(match['frames'][frame]['participantFrames'][str(i)]['xp'])
        current_gold.append(frame_current_gold)
        total_gold.append(frame_total_gold)
        xp.append(frame_xp)
        
    team_current_gold = calculate_team_total_and_difference(np.array(current_gold), frames)
    team_total_gold = calculate_team_total_and_difference(np.array(total_gold), frames)
    team_xp = calculate_team_total_and_difference(np.array(xp), frames)
    
    team_max_current_gold = calculate_team_max_and_difference(np.array(current_gold), frames)
    team_max_total_gold = calculate_team_max_and_difference(np.array(total_gold), frames)
    team_max_xp = calculate_team_max_and_difference(np.array(xp), frames)
    
    return team_current_gold, team_total_gold, team_xp, team_max_current_gold, team_max_total_gold, team_max_xp, frames

team_stats_vars = ['firstBlood','firstTower','firstDragon','firstRiftHerald','firstInhibitor','firstBaron']

# inhibitor is well inside base; late-game predictor
# baron spawns at 20:00
# dragon spawns at 2:30 (http://leagueoflegends.wikia.com/wiki/Dragon)
# rift herald spawns at 9:50 (http://leagueoflegends.wikia.com/wiki/Rift_Herald)
# baron spawns at 20:00 http://leagueoflegends.wikia.com/wiki/Baron_Nashor

def get_team_stats(match):
    team_0_stats = np.array([1*match['teams'][0][var] for var in team_stats_vars])
    team_1_stats = np.array([1*match['teams'][1][var] for var in team_stats_vars])
    team_diff = team_0_stats - team_1_stats
    return team_diff

def convert_to_tensor(dat, num_matches, num_frames, max_frames):
    new_dat = []
    for i in range(num_matches):
        new_dat.append(np.pad(dat[i], ((0,max_frames - num_frames[i]), (0,0)), 'constant'))
    new_dat = np.array(new_dat)
    return new_dat

In [8]:
current_gold = []
total_gold = []
xp = []
max_current_gold = []
max_total_gold = []
max_xp = []
champions = []

winners = []
num_frames = []
match_ids = []
team_stats = []
duration = []
player_tiers = []
versions = []
game_types = []

for i, file in enumerate(files):
    if i%500 == 0:
        print(i)
        
    with open(file, 'r') as fp:
        
        match_file = file.replace('timeline','match')

        with open(match_file, 'r') as fp2:
            match2 = json.load(fp2)
            if match2['mapId'] != 11: # summoner's rift only
                continue
            n = len(match2['participants'])
            if n != 10:
                continue
            current_champion_ids = [match2['participants'][participant_id]['championId'] for participant_id in range(10)]
            if len(np.unique(current_champion_ids)) < 10:
                continue
        
        match = json.load(fp)
            
        try:
            team_current_gold, team_total_gold, team_xp, team_max_current_gold, team_max_total_gold, team_max_xp, frames = get_dat(match)
            current_tiers = [match2['participants'][i]['highestAchievedSeasonTier'] for i in range(10)]
        except:
            continue
       
        match_id = match2['gameId']
        
        versions.append(match2['gameVersion'])
        game_types.append(match2['gameType'])
        player_tiers.append(current_tiers)
        
        champions.append(current_champion_ids)
        
        duration.append(match2['gameDuration'])
        
        winner = (match2['teams'][0]['win'] == 'Fail')*1
        winners.append(winner)
        match_ids.append(match_id)
        
        max_current_gold.append(team_max_current_gold)
        max_total_gold.append(team_max_total_gold)
        max_xp.append(team_max_xp)

        current_gold.append(team_current_gold)
        total_gold.append(team_total_gold)
        xp.append(team_xp)
        num_frames.append(frames)
        
        cur_team_stats = get_team_stats(match2)
        team_stats.append(cur_team_stats)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [9]:
num_matches = len(winners)
print(num_matches)

max_frames = np.max(num_frames)
print(max_frames)

7302
72


In [10]:
tensor_current_gold = convert_to_tensor(current_gold, num_matches, num_frames, max_frames)
tensor_total_gold = convert_to_tensor(total_gold, num_matches, num_frames, max_frames)
tensor_xp = convert_to_tensor(xp, num_matches, num_frames, max_frames)
tensor_max_current_gold = convert_to_tensor(max_current_gold, num_matches, num_frames, max_frames)
tensor_max_total_gold = convert_to_tensor(max_total_gold, num_matches, num_frames, max_frames)
tensor_max_xp = convert_to_tensor(max_xp, num_matches, num_frames, max_frames)
tensor_team_stats = np.array(team_stats)
winners = np.array(winners)
num_frames = np.array(num_frames)
champions = np.array(champions)
duration = np.array(duration)
versions = np.array(versions)
game_types = np.array(game_types)

player_tiers = np.array(player_tiers)
map_tiers = {
    'UNRANKED': 0,
    'BRONZE': 1,
    'SILVER': 2,
    'GOLD': 3,
    'PLATINUM': 4,
    'DIAMOND': 5,
    'MASTER': 6,
    'CHALLENGER': 7
}
player_tiers = np.vectorize(map_tiers.get)(player_tiers)

In [11]:
print(tensor_current_gold.shape)
print(tensor_total_gold.shape)
print(tensor_xp.shape)
print(tensor_max_current_gold.shape)
print(tensor_max_total_gold.shape)
print(tensor_max_xp.shape)
print(tensor_team_stats.shape)
print(winners.shape)
print(num_frames.shape)
print(champions.shape)
print(duration.shape)
print(versions.shape)
print(game_types.shape)
print(player_tiers.shape)

(7302, 72, 3)
(7302, 72, 3)
(7302, 72, 3)
(7302, 72, 3)
(7302, 72, 3)
(7302, 72, 3)
(7302, 6)
(7302,)
(7302,)
(7302, 10)
(7302,)
(7302,)
(7302,)
(7302, 10)


In [12]:
# SAVING DATA

#fout = 'lol_dat.npz'
fout = 'lol_dat_challenger.npz'

np.savez(fout, current_gold = tensor_current_gold, total_gold = tensor_total_gold, xp = tensor_xp,
        max_current_gold = tensor_max_current_gold, max_total_gold = tensor_max_total_gold, max_xp = tensor_max_xp,
        team_stats = tensor_team_stats, winners = winners, num_frames = num_frames, champions = champions, duration = duration,
        versions = versions, game_types = game_types, player_tiers = player_tiers)

In [25]:
# LOADING DATA

npzfile = np.load(fout)
npzfile.files

['current_gold',
 'total_gold',
 'xp',
 'max_current_gold',
 'max_total_gold',
 'max_xp',
 'team_stats',
 'winners',
 'num_frames',
 'champions']

In [54]:
match2.keys()

dict_keys(['gameId', 'platformId', 'gameCreation', 'gameDuration', 'queueId', 'mapId', 'seasonId', 'gameVersion', 'gameMode', 'gameType', 'teams', 'participants', 'participantIdentities'])

In [103]:
tiers = np.unique(player_tiers)



['BRONZE' 'CHALLENGER' 'DIAMOND' 'GOLD' 'MASTER' 'PLATINUM' 'SILVER'
 'UNRANKED']


In [81]:
min_tier = 'PLATINUM'
min_ranking = map_tiers[min_tier]

fraction_cutoff = 7 # require this number of players to have min_ranking

valid_games = (np.sum(np.vectorize(map_tiers.get)(versions)>min_ranking,axis=1) >= fraction_cutoff)*1

In [85]:
match2['gameType']

'MATCHED_GAME'

In [96]:
for i in range(10):
    print(i,match2['participants'][i]['highestAchievedSeasonTier'])

0 UNRANKED
1 PLATINUM
2 UNRANKED
3 UNRANKED
4 UNRANKED


KeyError: 'highestAchievedSeasonTier'

In [141]:
all_events = []
all_victims = []

for f in range(frames):
    cur_events = match['frames'][f]['events']
    num_events = len(cur_events)
    team1_kills = 0
    team2_kills = 0
    for i in range(num_events):
        if cur_events[i]['type'] == 'CHAMPION_KILL':
            if cur_events[i]['victimId'] > 5:
                team1_kills += 1
            else:
                team2_kills += 1
    all_victims.append([team1_kills, team2_kills])
print(np.array(all_victims))

[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [3 1]
 [1 0]
 [0 1]
 [2 1]
 [1 2]
 [2 4]
 [0 1]
 [1 2]
 [0 1]
 [1 1]
 [0 3]
 [0 2]
 [0 0]
 [0 2]
 [3 4]
 [0 0]
 [1 0]
 [2 4]
 [1 0]
 [3 1]
 [4 0]
 [0 0]
 [4 0]
 [1 0]]


In [140]:
match2['teams']

[{'teamId': 100,
  'win': 'Win',
  'firstBlood': True,
  'firstTower': True,
  'firstInhibitor': True,
  'firstBaron': True,
  'firstDragon': False,
  'firstRiftHerald': False,
  'towerKills': 7,
  'inhibitorKills': 1,
  'baronKills': 1,
  'dragonKills': 0,
  'vilemawKills': 0,
  'riftHeraldKills': 0,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 16, 'pickTurn': 1},
   {'championId': 163, 'pickTurn': 2},
   {'championId': 121, 'pickTurn': 3},
   {'championId': -1, 'pickTurn': 4},
   {'championId': 107, 'pickTurn': 5}]},
 {'teamId': 200,
  'win': 'Fail',
  'firstBlood': False,
  'firstTower': False,
  'firstInhibitor': False,
  'firstBaron': False,
  'firstDragon': True,
  'firstRiftHerald': True,
  'towerKills': 6,
  'inhibitorKills': 0,
  'baronKills': 0,
  'dragonKills': 2,
  'vilemawKills': 0,
  'riftHeraldKills': 1,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 41, 'pickTurn': 6},
   {'championId': 119, 'pickTurn': 7},
   {'championId': 84, 'pickTurn': 8},
   {'ch